In [ ]:
import pandas as pd
import pydbtools as pydb # see https://github.com/moj-analytical-services/pydbtools
import altair as alt # for plotting, see https://altair-viz.github.io/getting_started/overview.html
import boto3

# few things for viewing dataframes better
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 900)
pd.set_option('display.max_colwidth', 200)

In [ ]:

# define some variables to be used throughout the notebook
database = "familyman_dev_v2"
snapshot_date = "2021-08-19"
database_derived = "familyman_derived_dev_v1"

In [ ]:
# This is adapted from one of the queries sent through
s1 = f"""
DROP TABLE ADOPTIONS_PARTIES;
"""

s2 = f"""
SELECT R.CASE_NUMBER,
  R.ROLE,
  R.ROLE_MODEL,
  R.REPRESENTATIVE_ROLE AS Representative,
  F.FIELD_MODEL,
  F.VALUE AS Rel_to_child,
  R.PARTY,
  P.DOB,
  P.GENDER,
  P.MOJAP_SNAPSHOT_DATE
FROM {database}.parties P
INNER JOIN {database}.roles R
ON P.PARTY = R.PARTY
INNER JOIN {database}.role_fields F
ON R.ROLE               = F.ROLE
WHERE ((R.ROLE_MODEL     = 'APLZ'
AND F.FIELD_MODEL = 'APLZ_RTC')
OR (R.ROLE_MODEL        = 'CHLDZ'
AND F.FIELD_MODEL = 'CHLDZ_CP'))
AND R.mojap_snapshot_date = date '{snapshot_date}'
AND F.mojap_snapshot_date = date '{snapshot_date}'
AND P.mojap_snapshot_date = date '{snapshot_date}'
"""

# Here we are creating a temporary table that can be queried later
# from the __temp__ database

table = pydb.create_temp_table(s2, "adoptions_parties")

# Query the temporary table just created. The database to query is called __temp__, this is
# an alias for a sandbox database that is created for each user. For more details, see
# the pydbtools docs
table = pydb.read_sql_query("select * from __temp__.adoptions_parties")
temp_table = pydb.read_sql_query("select count(*) as count from __temp__.adoptions_parties")

type(table)


In [ ]:
count_csv_file = temp_table.to_csv("s3://alpha-family-data/fcsq_processing/Adoption/adoption_count.csv")
adoptions_parties_csv_file = table.to_csv("s3://alpha-family-data/fcsq_processing/Adoption/adoptions_parties.csv")

In [ ]:
import boto3
s3 = boto3.resource('s3')

In [ ]:
pd.read_csv('s3://alpha-family-data/fcsq_processing/Adoption/adoption_count.csv')